# Quantum Imaginary Time Evolution (QITE)

**Table of Contents**

- [Introduction](#Introduction)
- [Hamiltonian and Initial State](#Hamiltonian-and-Initial-State)
- [QITE](#QITE)


## Introduction

In this tutorial, we will introduce how to simulate the imaginary-time evolution of quantum systems. The approach is grounded in a quantum algorithm that can efficiently model many-body quantum systems and approximate their ground state energies within polynomial time. Specifically, our focus lies on finding the ground state of a quantum system through Imaginary Time Evolution (ITE). This process is not only crucial for understanding quantum phase transitions but also plays a pivotal role in solving NP-hard problems such as the 3D Ising model and QMA-complete problems like those described by the Heisenberg model.

The imaginary-time evolution of quantum many-body systems is governed by the imaginary-time Schrödinger equation: 
$$
\partial_\tau|\phi(\tau)\rangle = -H|\phi(\tau)\rangle
,$$
where $\tau$ denotes imaginary time and $H$ is the time-independent Hamiltonian with an initial state $|\phi\rangle=|\phi(0)\rangle$. Our goal is to prepare a normalized imaginary-time evolved state 
$$
|\phi(\tau)\rangle=e^{-\tau H}|\phi\rangle / \|e^{-\tau H}|\phi\rangle\|
$$
on a quantum computer—a process known as ITE operation. For sufficiently large $\tau$, the system state typically converges to the ground state within the subspace induced by the initial state, often coinciding with the ground state of the Hamiltonian.

In [ ]:
import numpy as np
import torch

import quairkit as qkit
from quairkit import to_state, Hamiltonian
from quairkit.database import *
from quairkit.qinfo import *

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from qsp import *
from qite import *

import warnings

revise_tol(1e-40)

qkit.set_dtype('complex128')

## Hamiltonian and Initial State

To streamline the analysis, we make the following assumptions regarding the system's Hamiltonian and initial state:

1. **Hamiltonian Form**: The Hamiltonian $H$ consists of a linear combination of Pauli operators, i.e., $H=\sum_j h_j\sigma_j$, where $h_j$ are real coefficients. By appropriately rescaling and shifting the Hamiltonian, we ensure its eigenvalues lie within the interval $[-1, 1]$, with the ground state energy $\lambda_0$ being negative.
2. **Non-zero Spectral Gap**: We assume there exists a strictly positive spectral gap $\Delta=\lambda_1-\lambda_0>0$, indicating a unique ground state.
3. **Sufficient Imaginary Time Parameter**: The imaginary time parameter $\tau$ must be sufficiently large to satisfy certain polynomial conditions such as $|\lambda_0| \cdot poly(\tau) \gg 1$, $|\gamma|^2 \cdot poly(\tau) \gg 1$, and $\Delta \cdot \tau \gg 1$, where $\gamma=\langle\psi_0|\phi\rangle$ represents the overlap between the initial state $|\phi\rangle$ and the ground state $|\psi_0\rangle$.
4. **Positive Overlap Parameter**: The overlap parameter $\gamma=\langle\psi_0|\phi\rangle$ is positive, ensuring that the initial state contains components of the ground state.
5. **Finite Copies of Initial State**: Access to a finite number of copies of the initial state $|\phi\rangle$ is assumed.

These assumptions ensure the feasibility and theoretical foundation of the algorithm while simplifying practical implementation complexities. In the following sections, we will detail how to leverage these assumptions and background knowledge to write relevant code for simulating imaginary-time evolution of quantum systems.

The hamiltonian we used in experiment is an antiferromagnetic Heisenberg model for a 4-qubit homogeneous chain:
$$
H \propto \frac{1}{4} \sum_{j=1}^3 (X_j X_{j+1}+Y_j Y_{j+1}+Z_j Z_{j+1} - I^{\otimes 4})
$$

Get normalized Hamiltonian and initial state.

In [ ]:
num_qubits = 5

H_init = afm_heisenberg(n=num_qubits)
H = normalize(H_init, factor=np.abs(H_init.coefficients).sum())  # normalize the Hamiltonian

In [ ]:
eigenvalues, eigenvectors = torch.linalg.eigh(H.matrix)

min_eigen = eigenvalues.real.min()
min_index = torch.argmin(min_eigen)

min_eigenvector = eigenvectors[:, min_index]
print(f'min_eigenvector = {min_eigenvector}')
print(f'min_eigenvalue = {min_eigen}')

min_eigenvector_state = to_state(min_eigenvector)

min_eigenvector = tensor([ 0.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j, -4.8378e-17+0.j,
         0.0000e+00+0.j, -2.5674e-16+0.j,  1.3497e-16+0.j, -4.7989e-02+0.j,
         0.0000e+00+0.j,  2.1119e-16+0.j, -2.2204e-16+0.j,  2.3302e-01+0.j,
         3.0296e-16+0.j, -3.8445e-01+0.j,  1.9941e-01+0.j,  2.9297e-17+0.j,
         0.0000e+00+0.j, -2.0894e-16+0.j,  3.2263e-16+0.j, -2.3302e-01+0.j,
        -6.3735e-18+0.j,  6.6546e-01+0.j, -3.8445e-01+0.j,  1.9109e-18+0.j,
        -1.3184e-16+0.j, -2.3302e-01+0.j,  2.3302e-01+0.j, -6.5052e-19+0.j,
        -4.7989e-02+0.j,  1.5416e-19+0.j, -3.2611e-20+0.j,  0.0000e+00+0.j])
min_eigenvalue = -0.7319715633294984


In [ ]:
small_index = torch.argmin(torch.where(torch.abs(min_eigenvector) > 1e-10, torch.abs(min_eigenvector), 1)).item()
phi = computational_state(index=small_index, system_dim=2 ** num_qubits)

gamma = torch.abs(phi.bra @ min_eigenvector_state.ket).squeeze()
print(f'gamma = {gamma}')

gamma = 0.04798860730154218


## QITE

Set initial data for QITE: $\tau$ and deg.

In [ ]:
tau = 35
deg = 672

We have set up two interfaces. The first one is `only_P`; if it is set to `True`, the simulation of QITE will be directly carried out using the Laurent polynomial, otherwise, the QSP simulation will be used. The second interface is `learn`, which allows us to learn and further optimize the angles in QPP.

In [ ]:
learn = False

Using QPE to get normalized lambda value within $1 / \tau$. Here, we directly provide this value and get the QPP angle. 

In [ ]:
normalized_lambda = -min_eigen.item() + 1 / tau

list_theta, list_phi = get_qpp_angle(guess_lambda=normalized_lambda, tau=tau, deg=deg, learn=learn)

Using the oracle to get the QPP circuit. Here, we used the ideal evolution operator instead of Trotter form.

In [ ]:
U = torch.matrix_exp(-1j * H.matrix)
cir = qpp_cir(list_theta, list_phi, U)

Considering the ancilla qubit, we have output state. The projection probability is the probability of measuring the ancilla qubit in the $|0\rangle$ state after applying the quantum circuit. In other words, it corresponds to the squared norm of the component of the output state where the ancilla is in the $|0\rangle$ state.

In [ ]:
cir.collapse([0], post_selection=0, if_print=True)

In [ ]:
input_state = nkron(zero_state(1), phi)
output_state = cir(input_state)

systems [0] collapse to the state |0> with (average) probability 0.00022656460353962735


The final normalized output state is obtained by dividing the output state by its norm.

In [ ]:
fidelity = state_fidelity(output_state.trace(0), min_eigenvector_state).item()
print(f'fidelity = {fidelity}')

fidelity = 0.9962053947472925


---

In [ ]:
qkit.print_info()


---------VERSION---------
quairkit: 0.4.3
torch: 2.8.0+cu128
torch cuda: 12.8
numpy: 2.3.3
scipy: 1.16.2
matplotlib: 3.10.6
---------SYSTEM---------
Python version: 3.13.7
OS: Linux
OS version: #63-Ubuntu SMP PREEMPT_DYNAMIC Tue Apr 15 19:04:15 UTC 2025
---------DEVICE---------
CPU:  AMD EPYC 9654 96-Core Processor
GPU: (0) NVIDIA GeForce RTX 4090
     (0) NVIDIA GeForce RTX 4090
